In [15]:
import sys
!conda install --yes --prefix {sys.prefix} numpy as np
import math as m
import sys
!conda install --yes --prefix {sys.prefix} matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact_manual

'conda' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
def coordinateImport():
    '''
    This function does not take inputs, but reads a text file with coordinates of the track, with the first column the
    x coordinates and the second, the y coordinates. The function output are two arrays of coordinates (x and y) which
    are reversed.
    '''
    trackcoordinates = open('skidpan.txt', 'r')  # open file to read
    #set empty array
    xArr = np.array([])
    yArr = np.array([])

    for line in trackcoordinates:
        xcoord, ycoord = [float(a) for a in line.strip().split()]
        xArr = np.append(xArr, xcoord)
        yArr = np.append(yArr, ycoord)

    # reverse arrays so that the track calculations are reversed
    xCoords = xArr[::-1]
    yCoords = yArr[::-1]
    return xCoords, yCoords


def effectiveCurve(xCoords, yCoords):
    '''
    The curvecalculator function takes the x and y coordinate arrays as inputs and creates initial and final values for
    the effective radius between three points. The for loop creates the effective radius for all other values in the
    track coordinates. The array of effective radii for each point on the track coordinates is output.
    '''
    # initial length values using the first two points and the last point.
    a0 = ((xCoords[1]-xCoords[-1])**2 + (yCoords[1] - yCoords[-1])**2)**0.5
    b0 = ((xCoords[0]-xCoords[-1])**2 + (yCoords[0] - yCoords[-1])**2)**0.5
    c0 = ((xCoords[1]-xCoords[0])**2 + (yCoords[1] - yCoords[0])**2)**0.5
    effectiveRadiusArr = np.array([])

    if c0 + b0 == a0:  # if the value of c0 + b0 is equal to a0 all three points are in a straight line, therefore
        # recorded as string 'straight'
        effectiveRadius0 = 'straight'
        effectiveRadiusArr = np.append(effectiveRadiusArr,effectiveRadius0)
    else:  # if they are not in a straight line, then the effective radius of the three points is estimated using trig,
        # cosine rule and circle theorem mathematics
        effectiveRadius0 = a0/(2*(np.sin(m.pi - abs(np.arccos(abs(((b0**2)+(c0**2)-(a0**2))/(2*b0*c0))))*180/m.pi)))
        effectiveRadiusArr = np.append(effectiveRadiusArr, effectiveRadius0)

    for i in range(1, len(xCoords)-1):
        # loop for points i-1,i and i+1 (outside the first and final positions).
        a = ((xCoords[i+1]-xCoords[i-1])**2 + (yCoords[i+1] - yCoords[i-1])**2)**0.5
        b = ((xCoords[i]-xCoords[i-1])**2 + (yCoords[i] - yCoords[i-1])**2)**0.5
        c = ((xCoords[i+1]-xCoords[i])**2 + (yCoords[i+1] - yCoords[i])**2)**0.5
        if b + c == a:  # if the value of c0 + b0 is equal to a0 all three points are in a straight line, therefore
        # recorded as string 'straight'
            effectiveRadius = 'straight'
            effectiveRadiusArr = np.append(effectiveRadiusArr,effectiveRadius)
        else:  # if they are not in a straight line, then the effective radius of the three points is estimated using trig,
        # cosine rule and circle theorem mathematics
            intstepresult = abs(((b**2)+(c**2)-(a**2))/(2*b*c))
            thirdstep = a/(2*np.sin(m.pi - abs(np.arccos(intstepresult)))*180/m.pi)
            effectiveRadius = a/(2*(np.sin(180 - abs(np.arccos(abs(intstepresult)*(m.pi/180))))*(m.pi/180)))
            effectiveRadiusArr = np.append(effectiveRadiusArr, effectiveRadius)
    # final length values using the first point and the two last points.
    aend = ((xCoords[0]-xCoords[-2])**2 + (yCoords[0] - yCoords[-2])**2)**0.5
    bend = ((xCoords[-1]-xCoords[-2])**2 + (yCoords[-1] - yCoords[-2])**2)**0.5
    cend = ((xCoords[0]-xCoords[-1])**2 + (yCoords[0] - yCoords[-1])**2)**0.5
    if cend + bend == aend:  # if the value of c0 + b0 is equal to a0 all three points are in a straight line, therefore
        # recorded as string 'straight'
        effectiveRadiusEnd = 'straight'
        effectiveRadiusArr = np.append(effectiveRadiusArr, effectiveRadiusEnd)
    else:  # if they are not in a straight line, then the effective radius of the three points is estimated using trig,
        # cosine rule and circle theorem mathematics
        effectiveRadiusEnd = aend/(2*(np.sin(m.pi - np.arccos(((bend**2)+(cend**2)-(aend**2))/(2*bend*cend))))*(180/m.pi))
        effectiveRadiusArr = np.append(effectiveRadiusArr, effectiveRadiusEnd)
    return effectiveRadiusArr

In [3]:
def PowerCurve():
    '''
    This function takes no inputs, but reads a text file of coordinates of Engine Horsepower in one column, and Engine
    RPM in the second column. The function also additionally calculates the Engine Torque from the horsepower. The
    function returns three arrays: power array, RPM array and the torque array.
    '''
    HpRPM = open('2017engine.txt', 'r')  # text file to read
    # set up empty arrays
    PowerArr = np.array([])
    RPMArr = np.array([])
    HpArr = np.array([])
    TorqueArray = np.array([])

    for line in HpRPM:
        RPM, Hp = [float(a) for a in line.strip().split()]
        RPMArr = np.append(RPMArr, RPM)
        HpArr = np.append(HpArr, Hp)
        PowerArr = np.append(PowerArr, Hp)
    #  for loop to calculate Torque at each RPM
    for i in range(0, len(RPMArr)):
        TorqueNewtonMeters = (HpArr[i]*5252)/(RPMArr[i])
        TorqueArray = np.append(TorqueArray, TorqueNewtonMeters)

    return PowerArr, RPMArr, TorqueArray

In [4]:
def enginePerformance(PowerArray, RPMArray, TorqueArray, finalDriveRatio, wheelDiameter):
    '''
    the function gearGraph takes a power array, RPM array and torque array as inputs. the function also opens and reads
    a text file consisting of the gear ratios, the RPM of the gear change from the given gear. A function of
    gearCalculation is called for each gear. The results are then plotted, with the output and array with 5 columns:
    Power, RPM, Velocity, Torque, Gear Ratio.
    '''
    gearRatios = open('gearratios.txt', 'r')  # file to read
    # set up empty arrays
    gearRatioArr = np.array([])
    gearChangeArr = np.array([])
    RPMGearVelArr = np.array([])
    # startLen is set to 0
    startLen = 0

    for line in gearRatios:  # strip and splitting the values in the file to read
        gearRatio, gearChange = [float(a) for a in line.strip().split()]
        gearRatioArr = np.append(gearRatioArr, gearRatio)
        gearChangeArr = np.append(gearChangeArr, gearChange)
    for n in range(0, len(gearRatioArr)):  # for loop for each gear, to output the 5 column array
        a = gearCalculation(PowerArray, RPMArray, TorqueArray, finalDriveRatio, wheelDiameter, gearRatioArr[n], gearChangeArr[n], startLen)
        RPMGearVelArr = np.append(RPMGearVelArr, a[0])
        startLen = a[1]  # set new startLen
    RPMGearVelArr = np.reshape(RPMGearVelArr, (int(len(RPMGearVelArr)/5), 5))  # reshape array to have 5 columns

    return RPMGearVelArr

In [5]:
def gearCalculation(PowerArray, RPMArray, TorqueArray, finalDriveRatio, wheelDiameter, gearRatio, gearChange, startLen):
    '''
    gearCalculation takes inputs of a power array, rpm array, torque array, a final drive ratio, the wheel diameter,
    gear ratio array, gear change array and a start length (therefore when a gear change occurs the rpm is not reset to
    the lowest value). The function outputs an array with 5 columns: Power, RPM, Velocity, Torque, Gear Ratio and a
    startLen value for the next iteration to start on the same rpm.
    '''

    gearArr = np.array([])
    for a in range(startLen, len(RPMArray)):  # for loop from the starLen (governed by previous gear change) and the end
        #  of the RPM array
        gearVel = ((((RPMArray[a]/60)/gearRatio)/finalDriveRatio)/(m.pi*wheelDiameter))  # calculation for the velocity
        # at the particular gear ratio
        ArrOne = np.array([PowerArray[a], RPMArray[a], gearVel, TorqueArray[a], gearRatio])  # set up array with initial values
        gearArr = np.append(gearArr, ArrOne)
        if RPMArray[a] >= gearChange:  # if the current RPM is greater than or equal to the gear change value, return
            # formatted array
            fifthLengthLast = startLen  # previous startLen is saved
            fifthLength = int(len(gearArr)/5)
            gearArr = np.reshape(gearArr, (fifthLength, 5))  # reshape the array into 5 columns
            startLen = fifthLength+fifthLengthLast  # new startLen is equal to the length of the array looped plus
            # the previous startLen value
            return gearArr, startLen  # output the array and the value of startLen
    fifthLength = int(len(gearArr)/5)
    gearArr = np.reshape(gearArr, (fifthLength, 5))  # reshape the array into 5 columns
    startLen = fifthLength+startLen

    return gearArr, startLen  # output the array and the value of startLen

In [6]:
def vehicleValues():
    '''
    This function reads a text file with various vehicle parameters. It returns the following values: tyre friction
    coefficient, mass, air density, frontal area, drag coefficient, efficiency, acceleration due to gravity, rolling
    resistance coefficient, area of lift, final drive ratio, wheel diameter.
    '''
    values = open('vehiclevalues.txt', 'r')  # open file to read
    for line in values:  # reading and assigning values
        tyreFrictionCoeff, mass, density, frontalArea, dragCoefficient, efficiency, g, rollingResistanceCoefficient, \
        liftArea, liftCoefficient, finalDriveRatio, wheelDiameter = [float(a) for a in line.strip().split()]
    return tyreFrictionCoeff, mass, density, frontalArea, dragCoefficient, efficiency, g, rollingResistanceCoefficient,\
           liftArea, liftCoefficient, finalDriveRatio, wheelDiameter

In [7]:
def maxCornerVel(tyreFrictionCoeff, normalForce, mass, effectiveRadius, airDen, Area, cd):
    '''
    The maxCornerVel function takes inputs of the tyre friction coefficient, the normal Force acting on the vehicle,
    the mass of the vehicle, the effective radius, the air density, the frontal area and the drag coefficient. The
    function returns a value of velocity which represents the maximum the vehicle can navigate the corner.
    '''
    vel = (((tyreFrictionCoeff*normalForce)**2)/(mass/(float(effectiveRadius)**2)+((0.5*airDen*Area*cd))**2))**0.25
    return vel

In [8]:
def acceleration(Torque, gearRatio, finalDriveRatio, efficiency, wheelDiameter, density, dragCoefficient, frontalArea,
                 vel, mass, g, coeffRollingResistance):
    '''
    this function takes a torque values, a gear ratio, final drive ratio, the efficiency of the vehicle powertrain,
    wheel diameter, air density, drag coefficient, frontal area, velocity, mass, acceleration due to gravity and the
    coefficient of rolling resistance. The function returns an a value of acceleration.
    '''
    accelForce = (Torque*gearRatio*finalDriveRatio*efficiency)/(wheelDiameter/2)
    aeroForce = (density*dragCoefficient*frontalArea*(vel**2))/2
    rollingResistance = mass*g*coeffRollingResistance

    a = (accelForce-aeroForce-rollingResistance)/mass
    return a

In [9]:
def normalForce(mass, g, density, cl, liftArea, vel):
    '''
    normalForce function takes inputs of mass, acceleration due to gravity, the lift coefficient(should be negative due
    to downforce, the area of lift, and the velocity. The function outputs a force value.
    '''
    F = float(mass*g - (density*cl*liftArea*(vel**2))/2)
    return F

In [10]:
def calculator(xCoords, yCoords, RPMGearVelArr, finalDriveRatio, wheelDiameter,efficiency,airDensity,
               dragCoefficient,frontalArea,mass, g,rollingResistanceCoefficient,tyreFrictionCoefficient,
               liftArea,liftCoefficient, initialVel):
    '''
    This function takes the array of x coordinates, array of y coordinates and an array with five columns of:
    Power, RPM, Velocity, Torque, Gear Ratio. The initial normal force is calculated using the maximum velocity of the
    engine performance. The for loop iterates over the trackcoordinates, calculating the acceleration from the previous
    point, with a limit on the speed due to the maximum corner velocity. Assuming the acceleration is constant over the
    distance between the points the velocity can be calculated. The output is the time of the lap and a velocity
    distance graph.
    '''

    effectiveRadiusArr = effectiveCurve(xCoords, yCoords)  # obtain track coordinates from Track script
    u = initialVel
    vArr = np.array([u])  # create velocity array with first value
    torqueInterpArr = np.array([0])  # create Torque array with 0 as the first value
    entranceVelArr = np.array([])  # create empty array for entrance velocity
    distArr = np.array([0])
    count = 0  # set count equal to 0
    for i in range(0, len(xCoords)-1):

        count = count+1  # increase count
        hyp = ((xCoords[i+1]-xCoords[i])**2 + (yCoords[i+1] - yCoords[i])**2)**0.5  # calculate hypotenuse of points
        dist = distArr[-1]+hyp  # cumulative addition of hyp to the distance
        distArr = np.append(distArr, dist)

        u = vArr[[-1]]  # u is equal to the final value in vArr
        torqueInterp = np.interp(u, RPMGearVelArr[:, 2], RPMGearVelArr[:, 3])  # interpolate the torque for the
        # given velocity
        torqueInterpArr = np.append(torqueInterpArr, torqueInterp)
        gearRatioInterp = np.interp(u, RPMGearVelArr[:, 2], RPMGearVelArr[:, 4])  # approximation of interpolating the
        # gear values
        # using function acceleration to calculate the acceleration
        a = acceleration(torqueInterp, gearRatioInterp, finalDriveRatio, efficiency, wheelDiameter, airDensity,
                         dragCoefficient, frontalArea, u, mass, g, rollingResistanceCoefficient)
        v = (u**2 + 2*hyp*a)**0.5  # assuming constant acceleration over the distance between the points
        if effectiveRadiusArr[i] != 'straight':  # if the effective radius is not 'straight', then the max corner
            # velocity may limit the velocity of the vehicle. If the velocity is larger than the maximum value it is
            # assumed the vehicle will travel at the maximum possible velocity
            float(effectiveRadiusArr[i])
            corner = maxCornerVel(tyreFrictionCoefficient, normalForce(mass, g, airDensity, liftCoefficient,
                                liftArea, u), mass, effectiveRadiusArr[i], airDensity, frontalArea, dragCoefficient)
            if v > corner:
                v = corner
        vArr = np.append(vArr, v)
        entranceVelArr = np.append(entranceVelArr, v)
    return distArr, vArr

In [11]:
def accelerationRestCalculator(xCoordsReversed, yCoordsReversed, RPMGearVelArr, finalDriveRatio, wheelDiameter,
                               efficiency,airDensity, dragCoefficient,frontalArea,mass, g,
                               rollingResistanceCoefficient,tyreFrictionCoefficient, liftArea,liftCoefficient):
    '''
    accelerationRestCalculator takes the array of x coordinates, array of y coordinates and an array with five columns
    of: Power, RPM, Velocity, Torque, Gear Ratio. The initial normal force is calculated using the current velocity of
    zero. The for loop iterates over the trackcoordinates (which have been reversed back to 'forward', calculating
    the acceleration from the previous point, with a limit on the speed due to the maximum corner velocity. Assuming
    the acceleration is constant over the distance between the points the velocity can be calculated. The output is the
    time of the lap and a velocity distance graph.
    '''
    xCoords = xCoordsReversed[::-1]  # reverse the input arrays so that they are 'forward'
    yCoords = yCoordsReversed[::-1]
    effectiveRadiusArr = effectiveCurve(xCoords, yCoords)
    u = 0  # initial velocity is set to zero
    vArr = np.array([u])  # create velocity array with an initial value (of 0)
    # create empty arrays
    torqueInterpArr = np.array([])
    entranceVelArr = np.array([])
    distArr = np.array([0])
    timeArr = np.array([0])

    for i in range(0, len(xCoords)-1):
        hyp = ((xCoords[i+1]-xCoords[i])**2 + (yCoords[i+1] - yCoords[i])**2)**0.5  # calculate hypotenuse
        dist = distArr[-1]+hyp  # cumulative addition of hyp to the distance
        distArr = np.append(distArr, dist)
        u = vArr[-1]  # u is equal to the final value in vArr
        torqueInterp = np.interp(u, RPMGearVelArr[:, 2], RPMGearVelArr[:, 3])  # interpolate the torque for the
        # given velocity
        gearRatioInterp = np.interp(u, RPMGearVelArr[:, 2], RPMGearVelArr[:, 4])  # approximation of interpolating the
        # gear values
        # using function acceleration to calculate the acceleration
        a = acceleration(torqueInterp, gearRatioInterp, finalDriveRatio, efficiency, wheelDiameter, airDensity,
                         dragCoefficient, frontalArea, u, mass, g, rollingResistanceCoefficient)
        v = (u**2 + 2*hyp*a)**0.5  # assuming constant acceleration over the distance between the points
        if effectiveRadiusArr[i] != 'straight':  # if the effective radius is not 'straight', then the max corner
            # velocity may limit the velocity of the vehicle. If the velocity is larger than the maximum value it is
            # assumed the vehicle will travel at the maximum possible velocity
            float(effectiveRadiusArr[i])
            corner = maxCornerVel(tyreFrictionCoefficient, normalForce(mass, g, airDensity, liftCoefficient,
                                liftArea, u), mass, effectiveRadiusArr[i], airDensity, frontalArea, dragCoefficient)
            if v > corner:
                v = corner

        vArr = np.append(vArr, v)
        torqueInterpArr = np.append(torqueInterpArr, torqueInterp)
        entranceVelArr = np.append(entranceVelArr, v)


    averageSpd = np.mean(vArr)
    return distArr, vArr

In [12]:
def join(reverseVel, accelDist, accelVel):
    '''
    The aim of this function is to produce a single array when the acceleration curve meets the reverse calculated
    array. The inputs of the function are: a velocity array of the reversed calculator, a distance array from the
    accelerating curve and a velocity array from the accelerating curve. The function then merges the two curves when
    the difference in the velocity is less than 1e-4. The function outputs a single distance array and a single
    velocity array.
    '''
    reverseVel = reverseVel[::-1]  # make the reverse velocity array forwards for comparison
    velocityArr = np.array([])
    for i in range(0, len(reverseVel)):
        bool = 0  # create boolean and set to 0
        if abs(reverseVel[i] - accelVel[i]) < 1e-4:  # determine whether the difference in arrays is within the
            # tolerance to join
            velocityArr = np.append(velocityArr, reverseVel[i])
            bool = 1
        elif bool == 1:
            velocityArr = np.append(velocityArr, reverseVel[i])
        else:
            velocityArr = np.append(velocityArr, accelVel[i])
    meanSpd = np.mean(abs(velocityArr))
    time = accelDist[-1]/meanSpd
    AA = len(accelDist)
    BB = len(velocityArr)
    # plt.plot(accelDist, velocityArr, color='blue')
    # plt.title('Distance Velocity graph')
    # plt.xlabel('distance/m')
    # plt.ylabel('velocity/ms^-1')
    # plt.show()
    return accelDist, velocityArr

In [13]:
def lapfunction(dist, vel, finalDriveRatio, wheelDiameter, efficiency,airDensity,dragCoefficient,frontalArea,mass, g,
                    rollingResistanceCoefficient, tyreFrictionCoefficient, liftArea, liftCoefficient, lapNumber):
    boolean = True  # boolean for error catch

#     while boolean:  # while loop for error catch
#         try:
#             lapNumber = int(input('Number of laps'))  # input number
#             if lapNumber <= 0:  # used to check if lapNumber is positive
#                 print('Number of laps must be a positive integer.')
#             else:  # exits while loop if satisfied
#                 boolean = False
#         except ValueError:  # ValueError raised
#             print('Number of laps must be a positive integer.')

    distArr = np.array(dist)
    velArr = np.array(vel)
    if lapNumber == 1:  # if only a single lap
        plt.plot(distArr, velArr)
        plt.show()
        return distArr, velArr
    else:  # if lap is greater than one append more laps to the existing array - using the latest value of
        # velocity for the initial velocity of each lap
        lapOneTime = distArr[-1]/np.mean(velArr)
        print('lap time for lap 1          : ', '{:.4f}'.format(lapOneTime), 'seconds')
        for n in range(0, lapNumber-1):
            calculatedArray = calculator(coordinateImport()[0], coordinateImport()[1],
        enginePerformance(PowerCurve()[0], PowerCurve()[1], PowerCurve()[2], finalDriveRatio, wheelDiameter),
                    finalDriveRatio, wheelDiameter,efficiency,airDensity,dragCoefficient,frontalArea,mass, g,
                    rollingResistanceCoefficient, tyreFrictionCoefficient, liftArea, liftCoefficient, velArr[-1])
            calculatedArray = calculatedArray[::-1]
            lapDist = calculatedArray[1][-1]
            lapTime = lapDist/np.mean(calculatedArray[0])
            dist = distArr[-1] + calculatedArray[1]

            distArr = np.append(distArr, dist)
            velArr = np.append(velArr, calculatedArray[0])
            if 9 < n+2 < 100:  # if loop for formatting depending on number of figures in the lap number
                print('lap time for lap', n+2, '        : ', '{:.6f}'.format(lapTime), 'seconds')
            elif n+2 < 10:
                print('lap time for lap', n+2, '         : ', '{:.6f}'.format(lapTime), 'seconds')
            else:
                print('lap time for lap', n+2, '       : ', '{:.6f}'.format(lapTime), 'seconds')
    aveSpd = np.mean(velArr)  # find average speed of vehicle
    totaldist = distArr[-1]  # obtain the total distance covered
    time = totaldist/aveSpd  # calculate time
    print('Average Velocity of all laps: ', '{:.6f}'.format(aveSpd), 'metres per second',
          '({:.6f}'.format(aveSpd*2.23694), 'mph)')
    print('Total Distance of all laps  : ', '{:.6f}'.format(totaldist), 'metres')
    return distArr, velArr

In [14]:
#parameter widget function
'''

'''
def parameters(tyreFrictionCoefficient, mass, airDensity, frontalArea, dragCoefficient, efficiency,
               rollingResistanceCoefficient, liftArea, liftCoefficient, finalDriveRatio,
               wheelDiameter, lapNumber):
    
    tyreFrictionCoefficient = float(tyreFrictionCoefficient)
    mass = float(mass)
    airDensity = float(airDensity)
    frontalArea = float(frontalArea)
    dragCoefficient = float(dragCoefficient)
    efficiency = float(efficiency)
    rollingResistanceCoefficient = float(rollingResistanceCoefficient)
    liftArea = float(liftArea)
    liftCoefficient = float(liftCoefficient)
    finalDriveRatio = float(finalDriveRatio)
    wheelDiameter = float(wheelDiameter)
    lapNumber = int(lapNumber)
    
    engineResults = enginePerformance(PowerCurve()[0], PowerCurve()[1], PowerCurve()[2],
                                         finalDriveRatio, wheelDiameter)
    fig, ax1 = plt.subplots(figsize=(10, 5))  # plot engine results (Power, RPM and Torque against velocity)

    ax1.set_xlabel('Velocity [m/s]')
    ax1.set_ylabel('Power [bhp]')
    plt.plot(engineResults[:, 2], engineResults[:, 0], color='tab:red',label='Power')
    plt.legend(loc='lower right')
    ax2 = ax1.twinx()
    ax2.set_ylabel('RPM [RPM]', color='tab:blue')
    plt.plot(engineResults[:, 2], engineResults[:, 1], color='tab:blue', label='RPM')
    ax2.tick_params(axis='y', labelcolor='tab:blue')
    plt.title('Engine Power and Engine RPM against Vehicle Velocity')
    plt.legend(loc='upper left')
    plt.show()

    plt.plot(engineResults[:, 2], engineResults[:, 3], color='tab:green')
    plt.title('Engine Torque against Vehicle Velocity')
    plt.ylabel('Torque [Nm]')
    plt.xlabel('Vehicle Velocity [m/s]')
    plt.show()

    coordinates = coordinateImport()  # plot track coordinates
    plt.plot(coordinates[0], coordinates[1])
    plt.xlabel('x')
    plt.axis('equal')
    plt.ylabel('y')
    plt.title('Track')
    plt.show()

    a = (calculator(coordinateImport()[0], coordinateImport()[1],
        enginePerformance(PowerCurve()[0], PowerCurve()[1], PowerCurve()[2], finalDriveRatio, wheelDiameter),
                    finalDriveRatio, wheelDiameter,efficiency,airDensity,dragCoefficient,frontalArea,mass, g,
                    rollingResistanceCoefficient, tyreFrictionCoefficient, liftArea, liftCoefficient, 25))

    b = (accelerationRestCalculator(coordinateImport()[0], coordinateImport()[1], enginePerformance
        (PowerCurve()[0], PowerCurve()[1], PowerCurve()[2], finalDriveRatio, wheelDiameter,),finalDriveRatio,
                                    wheelDiameter,efficiency,airDensity,dragCoefficient,frontalArea,mass, g,
                    rollingResistanceCoefficient, tyreFrictionCoefficient, liftArea, liftCoefficient))
    # join(a[1], b[0], b[1])
    totaldistandvel = lapfunction(join(a[1], b[0], b[1])[0], join(a[1], b[0], b[1])[1],finalDriveRatio, wheelDiameter,
                                  efficiency,airDensity,dragCoefficient,frontalArea,mass, g,
                    rollingResistanceCoefficient, tyreFrictionCoefficient, liftArea, liftCoefficient, lapNumber)
    aveSpd = np.mean(totaldistandvel[1])
    time = totaldistandvel[0][-1]/aveSpd
    print('Total Race Time             : ', '{:.6f}'.format(time), 'seconds')

    plt.plot(totaldistandvel[0], totaldistandvel[1])
    plt.title('Lap Simulator Total Distance against Velocity')
    plt.xlabel('Distance [m]')
    plt.ylabel('Velocity [m/s]')
    plt.show()
    
    #return tyreFrictionCoefficient, mass,airDensity,frontalArea, dragCoefficient, efficiency, rollingResistanceCoefficient, liftArea, liftCoefficient, finalDriveRatio, wheelDiameter


interact_manual(parameters, tyreFrictionCoefficient=widgets.FloatSlider(0.92,min=0.7,max=1.1,step=0.01),
                       mass = widgets.FloatSlider(320,min=200,max=500,step=1),
                        airDensity = widgets.FloatSlider(1.225,min=1.1455,max=1.2922,step=0.0001),
                        frontalArea = widgets.FloatSlider(0.83,min=0.5,max=2.0,step=0.01),
                        dragCoefficient = widgets.FloatSlider(0.7,min=0.2,max=1.5,step=0.01),
                        efficiency = widgets.FloatSlider(0.8,min=0.6,max=1,step=0.01),
                        rollingResistanceCoefficient = widgets.FloatSlider(0.0013,min=0.0005,max=0.01,step=0.0001),
                        liftArea = widgets.FloatSlider(0.7,min=0,max=2,step=0.1),
                        liftCoefficient = widgets.FloatSlider(-1.04,min=-1.5,max=0,step=0.01),
                        finalDriveRatio = widgets.FloatSlider(3.45,min=3,max=4,step=0.01),
                        wheelDiameter = widgets.FloatSlider(0.513,min=0.3,max=1,step=0.001),
                        lapNumber = widgets.IntSlider(1,min=1,max=50,step=1))

g=float(9.81)


interactive(children=(FloatSlider(value=0.92, description='tyreFrictionCoefficient', max=1.1, min=0.7, step=0.…